<a href="https://www.kaggle.com/code/kennethasmith/bias-breaker-ai-agent?scriptVersionId=287824078" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [11]:
#Import neccessary packages
import pandas as pd
import numpy as np
import json
import asyncio
from typing import Any, Dict, List
from google.genai import types
from google.adk.agents import LlmAgent, SequentialAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner  # Key change based on reference
from google.adk.sessions import InMemorySessionService # Key change based on reference
from google.adk.tools import FunctionTool

# --- 1. Custom Tool Function (Function remains the same) ---
def calculate_fairness_metrics(
    df_json: str, 
    protected_attribute: str,
    predictions_column: str,
    favorable_outcome: Any,
    unprivileged_group_value: Any,
) -> str:
    """
    Calculates key fairness metrics (e.g., Disparate Impact Ratio) and 
    runs a simulated feature contribution analysis.
    
    Args:
        df_json: JSON string of the dataset and predictions.
        protected_attribute: The column name for the protected group (e.g., 'gender').
        predictions_column: The column name containing the model's predictions.
        favorable_outcome: The value representing the positive outcome (e.g., 1 or 'Approved').
        unprivileged_group_value: The value representing the unprivileged group (e.g., 'Female').
        
    Returns:
        A comprehensive metrics and feature analysis report as a string.
    """
    try:
        # Load the data from the JSON string passed via the session
        df = pd.read_json(df_json)
        
        # --- Bias Metric Calculation: Disparate Impact Ratio (DIR) ---
        unprivileged_df = df[df[protected_attribute] == unprivileged_group_value]
        privileged_df = df[df[protected_attribute] != unprivileged_group_value]

        rate_unprivileged = (unprivileged_df[predictions_column] == favorable_outcome).mean()
        rate_privileged = (privileged_df[predictions_column] == favorable_outcome).mean()
        
        dir_value = rate_unprivileged / rate_privileged if rate_privileged != 0 else float('inf')

        # --- Proxy Bias/Feature Contribution Detection (Simulated for LLM analysis) ---
        feature_finding = ""
        if 'zip_code' in df.columns and dir_value < 0.8:
            feature_finding = "The 'zip_code' feature appears to be a strong **proxy for the protected attribute** (likely demographic data), contributing significantly to the disparate impact. It should be investigated for removal or masking."
        elif dir_value < 0.95:
             feature_finding = "Feature analysis suggests direct model dependence on the protected attribute itself. Re-weighting or pre-processing techniques are needed."
        else:
            feature_finding = "No clear proxy features detected, but the attribute itself is causing the disparity."
        
        # --- Format Report ---
        report = f"""
FAIRNESS ANALYSIS REPORT for Attribute: {protected_attribute}
==================================================

Primary Metric: Disparate Impact Ratio (DIR)
- Value: {dir_value:.4f}
- Violation Threshold: DIR below 0.8 or above 1.25 indicates significant bias.
- Bias Status: {'VIOLATION DETECTED' if dir_value < 0.8 or dir_value > 1.25 else 'PASS'}

Protected Group ({unprivileged_group_value}) Success Rate: {rate_unprivileged:.4f}
Reference Group Success Rate: {rate_privileged:.4f}

FEATURE CONTRIBUTION FINDING:
- Identified Contribution: {feature_finding}

Summary: The model exhibits **{('significant bias' if dir_value < 0.8 or dir_value > 1.25 else 'minor disparity')}** against the unprivileged group.
"""
        return report
        
    except Exception as e:
        return f"ERROR: Failed to run bias calculation: {e}"

# --- 2. Create the ADK Function Tool (CORRECTED) ---
fairness_tool = FunctionTool(calculate_fairness_metrics) # Pass the function directly!



In [8]:
#Bring in API Key
from kaggle_secrets import UserSecretsClient
import os

# This line fixes the "name 'os' is not defined" error
user_secrets = UserSecretsClient()
os.environ["GOOGLE_API_KEY"] = user_secrets.get_secret("GOOGLE_API_KEY")

print("✅ API Key successfully loaded into environment!")

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key configured successfully.")
except Exception as e:
    print(f"🔒 Authentication Error: {e}")
    print("Please add 'GOOGLE_API_KEY' to your Kaggle secrets.")

✅ API Key successfully loaded into environment!
✅ Gemini API key configured successfully.


In [12]:
#Import neccessary packages
import numpy as np
import json
import asyncio
from typing import Any, Dict, List
from google.genai import types
from google.adk.agents import LlmAgent, SequentialAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner  # Key change based on reference
from google.adk.sessions import InMemorySessionService # Key change based on reference
from google.adk.tools import FunctionTool

# Retry Config
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

# Models
GEMINI_FLASH = Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config)
GEMINI_PRO = Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config)

# Defining Memory Bank & Tools 

# We use a global dictionary to mimic the "Memory Bank" pattern
SESSION_MEMORY: Dict[str, Any] = {}

def save_report_tool(key: str, content: str) -> dict:
    """Tool to save reports/data to the global memory bank."""
    SESSION_MEMORY[key] = content
    return {"status": "success", "saved_key": key}

def get_report_tool(key: str) -> dict:
    """Tool to retrieve reports/data from the global memory bank."""
    data = SESSION_MEMORY.get(key)
    if data is None:
        return {"status": "error", "message": "Key not found"}
    return {"status": "success", "content": data}

# Custom Tool for Fairness Calculation
def calculate_fairness_metrics(
    df_json: str, 
    protected_attribute: str,
    predictions_column: str,
    favorable_outcome: int,
    unprivileged_group_value: str,
) -> str:
    """Calculates Disparate Impact Ratio from JSON data."""
    try:
        df = pd.read_json(df_json)
        
        # Calculate metrics
        unprivileged = df[df[protected_attribute] == unprivileged_group_value]
        privileged = df[df[protected_attribute] != unprivileged_group_value]
        
        rate_unpriv = (unprivileged[predictions_column] == favorable_outcome).mean()
        rate_priv = (privileged[predictions_column] == favorable_outcome).mean()
        
        if rate_priv == 0:
            dir_val = 0.0
        else:
            dir_val = rate_unpriv / rate_priv
            
        return json.dumps({
            "metric": "Disparate Impact Ratio",
            "value": dir_val,
            "status": "BIASED" if (dir_val < 0.8 or dir_val > 1.25) else "FAIR"
        })
    except Exception as e:
        return f"Error: {str(e)}"

# --- 4. Agent Definitions (Using LlmAgent like reference) ---

# Agent 1: DataPrepAgent
# Stores the raw data into the shared memory
data_prep_agent = LlmAgent(
    name="DataPrepAgent",
    model=GEMINI_FLASH,
    instruction="""
    You are a Data Ingestion Specialist.
    1. Accept the raw dataset JSON provided in the prompt.
    2. Call `save_report_tool` to save it under the key 'raw_data'.
    3. Confirm the data is saved and ready for analysis.
    """,
    tools=[save_report_tool]
)

# Agent 2: MetricsAgent
# Retrieves data, runs calculation, saves result
metrics_agent = LlmAgent(
    name="MetricsAgent",
    model=GEMINI_PRO,
    instruction="""
    You are an AI Metric Specialist.
    1. Retrieve 'raw_data' using `get_report_tool`.
    2. Use the `calculate_fairness_metrics` tool. 
       (Params: protected_attribute='gender', predictions_column='pred', favorable_outcome=1, unprivileged_group_value='Female')
    3. Call `save_report_tool` to save the tool's output JSON under the key 'metrics_result'.
    """,
    tools=[get_report_tool, save_report_tool, calculate_fairness_metrics]
)

# Agent 3: DiagnosisAgent
# Retrieves metrics, analyzes them
diagnosis_agent = LlmAgent(
    name="DiagnosisAgent",
    model=GEMINI_PRO,
    instruction="""
    You are a Fairness Diagnostician.
    1. Retrieve 'metrics_result' using `get_report_tool`.
    2. Analyze the Disparate Impact Ratio.
    3. If BIASED, explain WHY (e.g., historical bias in training data).
    4. Call `save_report_tool` to save your diagnosis text under 'diagnosis'.
    """,
    tools=[get_report_tool, save_report_tool]
)

# Agent 4: ScorecardAgent (Final Reporter)
scorecard_agent = LlmAgent(
    name="ScorecardAgent",
    model=GEMINI_FLASH,
    instruction="""
    You are the Final Report Generator.
    1. Retrieve 'metrics_result' and 'diagnosis' using `get_report_tool`.
    2. Generate a final Markdown 'Fairness Scorecard'.
    3. Output ONLY the Markdown report.
    """,
    tools=[get_report_tool]
)

# --- 5. Orchestration (SequentialAgent) ---

bias_breaker_pipeline = SequentialAgent(
    name="BiasBreakerPipeline",
    sub_agents=[
        data_prep_agent,
        metrics_agent,
        diagnosis_agent,
        scorecard_agent
    ],
)

# --- 6. Execution (Using InMemoryRunner) ---

# Create Runner 
runner = InMemoryRunner(bias_breaker_pipeline)

# Prepare Data
test_df = pd.DataFrame({
    'gender': ['Female', 'Male', 'Female', 'Male'] * 25,
    'pred': [0, 1, 0, 1] * 25 
})
dataset_json = test_df.to_json(orient='records')

# Initial Prompt containing the data
prompt = f"""
START ANALYSIS.
Here is the raw dataset JSON:
{dataset_json}
"""

print("🚀 Starting Bias Breaker Pipeline (Reference Architecture)...")

# Execute
# Note: In a notebook, you usually await this:
response = await runner.run_debug(prompt)

# Print Final Output
print("\n" + "="*50)
print("✅ FINAL AGENT OUTPUT")
print("="*50)

# Iterate through turns to find the final scorecard (similar to reference cell 10)
for turn in response:
    if getattr(turn, "source", "") == "ScorecardAgent":
        if hasattr(turn, "content") and turn.content and turn.content.text:
            print(turn.content.text)

🚀 Starting Bias Breaker Pipeline (Reference Architecture)...

 ### Created new session: debug_session_id

User > 
START ANALYSIS.
Here is the raw dataset JSON:
[{"gender":"Female","pred":0},{"gender":"Male","pred":1},{"gender":"Female","pred":0},{"gender":"Male","pred":1},{"gender":"Female","pred":0},{"gender":"Male","pred":1},{"gender":"Female","pred":0},{"gender":"Male","pred":1},{"gender":"Female","pred":0},{"gender":"Male","pred":1},{"gender":"Female","pred":0},{"gender":"Male","pred":1},{"gender":"Female","pred":0},{"gender":"Male","pred":1},{"gender":"Female","pred":0},{"gender":"Male","pred":1},{"gender":"Female","pred":0},{"gender":"Male","pred":1},{"gender":"Female","pred":0},{"gender":"Male","pred":1},{"gender":"Female","pred":0},{"gender":"Male","pred":1},{"gender":"Female","pred":0},{"gender":"Male","pred":1},{"gender":"Female","pred":0},{"gender":"Male","pred":1},{"gender":"Female","pred":0},{"gender":"Male","pred":1},{"gender":"Female","pred":0},{"gender":"Male","pred":1}

DataPrepAgent > The raw dataset has been successfully saved and is ready for analysis.


/tmp/ipykernel_47/3797910165.py:52: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(df_json)


MetricsAgent > The fairness metrics have been calculated and the report has been saved. The Disparate Impact Ratio is 0.0, which indicates that the model's predictions are BIASED against the unprivileged group ('Female').


DiagnosisAgent > The model's predictions are BIASED against the 'Female' group, as indicated by a Disparate Impact Ratio of 0.0. This suggests that the model is disproportionately unfavorable towards females in its predictions. This bias could stem from historical biases present in the training data, where past inequalities may have led to certain patterns that the model has learned and perpetuated.




ScorecardAgent > Hi, I'm the Final Report Generator. I will now generate the Fairness Scorecard based on the metrics and diagnosis.


✅ FINAL AGENT OUTPUT
